Заполните данные о необходимых переменных:

In [6]:
Today = '2024-06-30'
Mounth = 'Jun'

Print_ARAP = False
print_manual_map = False
print_counterclaims = True
excel_tofolder_on_Z = False

import os
from datetime import date
import pandas as pd
import numpy as np
import sys

sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

CCY_tech_dict = {'RUC':'CHF','RUE':'EUR','RUE2':'EUR','RUE3':'EUR','RUE5':'EUR','RUG':'GBP','RUJ':'JPY','RUK':'KZT','RUM':'MDL','RUU':'USD','RUU1':'USD','RUU1R':'USD','RUU2':'USD','RUU25':'USD','RUU26':'USD','RUU3':'USD','RUU5':'USD','RUY':'CNY','RUK5':'KZT','RUY3':'CNY','RUY5':'CNY'}

Открываю следующие файлы:

In [7]:
query = f'''select 
    "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding","currencyCode", "sourceSystem","presentAmountRUB",
CASE
        WHEN "bscCode" in ('130','135','BSC130','BSC135') 
        THEN 'Trade receivables'
        WHEN "bscCode" in ('160','165','BSC160','BSC165') 
        THEN 'Advances to suppliers'
        WHEN "bscCode" in ('170','175','BSC170','BSC175')
        THEN 'Capital advances'
        WHEN "bscCode" in ('140','145','BSC140','BSC145') 
        THEN 'Other receivables'
        WHEN "bscCode" in ('330','BSC330')
        THEN 'Provisions'
        WHEN "bscCode" in ('300','305','BSC300','BSC305') 
        THEN 'Trade payables'
        WHEN "bscCode" in ('310','315','BSC310','BSC315') 
        THEN 'Other payables'
        WHEN "bscCode" in ('360','365','BSC360','BSC365') 
        THEN 'Advances received'
        WHEN "bscCode" in ('350','355','BSC350','BSC355') 
        THEN 'Loans received'
        WHEN "bscCode" in ('210','215','BSC210','BSC215') 
        THEN 'Loans issued'
        WHEN "bscCode" in ('340','BSC340') 
        THEN 'Tax'
        WHEN "bscCode" in ('410','BSC410') 
        THEN 'Pension'
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    "sourceSystem" in ('OEBS12_EU','ETL document loader','SAP_HRG') and
    "rowType" = 'M' and
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')  
union all select
     "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding","currencyCode", "sourceSystem","presentAmountRUB",
CASE
        WHEN ("glAccount" like '62%' or "glAccount" IN (7699200310, 7699100210,4601000000, 4611000000, 1501010009, 1501010019, 1501010109, 1501010119, 2201010000,
        2201010008, 2201010009, 2201010010, 2201010018, 2201010019, 2201011000, 2201011008, 2201011009, 2201011010, 2201011018, 2201011019, 2201010098, 2201010099,
        2201020000, 2201020008, 2201030000, 2201030008, 2201010090, 2201010190, 2110000000, 2110000900, 2130000000, 2130000900, 2151000000, 2151000900, 2170000000,
        2170000900, 1210000000, 1210000900, 1260000000, 2181000000, 2181000900))
        THEN 'Trade receivables'
        WHEN "glAccount" IN (6002770900, 6002090900, 6002020000, 6002020099, 6002080000, 6002090000, 6002050000, 6002020900, 6002070000, 6002070099, 6002770000,
        6002050900, 6002070900, 6099000215, 6099000224, 6099000212, 6099000217, 6099000219, 6099000619, 6099000120, 6099000123, 6099000820, 6099000430, 6099000460,
        6099000470, 6099000490, 7699100410, 6099000830, 6099000810, 6002030000, 6002040000, 6002030900, 6002040900, 6099000440, 6099000450, 6099000203, 6099000615, 
        6002140000, 6002140099, 6099000616, 6002150000, 7610001900, 7610001000, 7610001099, 2301010100, 2301010108, 2301010110, 2301010118, 2301010200, 2301010208,
        2301010210, 2301010218, 2301010300, 2301010308, 2301010400, 2301010408, 2301011100, 2301011108, 2301011110, 2301011118, 2301011200, 2301011208, 2301011210,
        2301011218, 2301011300, 2301011308, 2302010000, 2302010007, 1611000000, 1613000000, 1615000000, 7610007900, 7610007000, 7610005000, 7699100620, 7699100420)
        THEN 'Advances to suppliers'
        WHEN "glAccount" IN (6002010000, 6002010099, 6002010900, 6099000612, 6099000211, 2301020100, 2301020108, 2301020110, 2301020118, 2301020200, 2301020208,
        2301020210, 2301020218, 2301020300, 2301020308, 2301020310, 2301020318, 2301021100, 2301021108, 2301021110, 2301021118, 2301021200, 2301021208, 2301021210,
        2301021218, 2301021300, 2301021308, 2301021310, 2301021318, 2301000008, 2301001008, 2302020000, 2302020007, 1612000000, 1614000000)
        THEN 'Capital advances'
        WHEN "glAccount" IN (6001089000, 7625003000, 7610003000, 7615003000, 7699100550, 6099000710, 7699200500, 7615004000, 7629009000, 7625013000, 7625013900,
        7699100200, 7629000900, 7610406000, 7610406900, 7610405000, 7610405900, 7630707000, 7622020000, 7620801000, 7620801900, 7630705000, 7629000099, 7629000000,
        7610004000, 7618000900, 7618000000, 6001080900, 6001080000, 7622010000, 7620703000, 7620704000, 7620701000, 7620701900, 7620702000, 7622030000, 7622040000,
        7699900000, 7610500000, 7610500099, 7699100240, 7630702000, 7630703000, 7630706000, 7625006000, 7625007000, 7625001000, 7625001900, 7620200000, 7620200900,
        2601030000, 2601030008, 2601040000, 2601040008, 2601050009, 2602000000, 2602000008, 2602000009, 2602000010, 2602000018, 2602000019, 2602001000, 2602001008,
        2602001010, 2602001018, 2602010000, 2602010009, 2602060100, 2602000090, 2603000000, 2603000100, 36307060, 1284000000, 1284000900, 1630000000, 1282000000, 
        1282000900, 1251000000, 2601010009, 2601020009)
        THEN 'Other receivables'
        WHEN "glAccount" IN (2201910000, 1290000000, 2190000000, 2301910100, 2602900000)
        THEN 'Provisions'
        WHEN ("glAccount" IN (1022200000))
        THEN 'FX Spot'
        WHEN ("glAccount" IN (1110009000, 1270000000, 1271000000, 2204000000, 2604010100, 2604020100, 2604021110, 2604021200, 7621156000))
        THEN 'Loans issued'
        WHEN ("glAccount" IN (1410000000, 1430010000, 1430030000, 1430050000, 1430060000, 1430070000))
        THEN 'Tax'
        WHEN ("glAccount" IN (2602000020))
        THEN 'MTM Derivatives'        
        WHEN ("glAccount" IN (7610300000))
        THEN 'Dividends'        
        WHEN ("glAccount" IN (7611102000))
        THEN 'Capital expenses'       
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'SAP ERP' or "sourceSystem" = 'SAP ECP') and
    "rowType" = 'M' and
    "reportType" = 'AR' and 
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')  
union all select
 "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding", "currencyCode", "sourceSystem","presentAmountRUB",
CASE
        WHEN "glAccount" like '63%'
        THEN 'Provisions'
        WHEN ("glAccount" IN (5201010100, 5201010110, 5201010200, 5201011100, 5201011110, 5201011200))
        THEN 'Trade payables'
        WHEN ("glAccount" IN (5501000000, 5501010000, 5502000000, 5503020000, 5503030000, 5503030010, 5503010000))
        THEN 'Other payables'
        WHEN ("glAccount" like '62%' or "glAccount" IN (5301010000, 5301010010, 7620002000, 0035100000))
        THEN 'Advances received'
        WHEN ("glAccount" like '60%' or "glAccount" IN (5101010100, 5101020110, 5101021110, 5104040000, 5109000000, 5503050000, 5505010200, 5505010210, 5505011210, 5505020100, 
        7610101000, 7610402000, 7610404000, 7610405000, 7610601000, 7610603000, 7610691000, 7619000000, 7620002000, 0033970000, 0033500000, 0033600000, 0033960000, 0033140000,
        0033130000, 0033120000, 0033110000, 0033800000, 0033950000, 0033970100))
        THEN 'Other payables'
        WHEN ("glAccount" IN (0032200000, 0032310000))
        THEN 'Pension'
        WHEN ("glAccount" IN (0032300000, 0032150000, 0032140000, 0032130000, 0032120000, 0032110000, 0031500000, 0033970200))
        THEN 'Social'
        WHEN ("glAccount" IN (0031940000, 0031930000, 0031920000, 0031910000, 0031800000, 0031700000, 0031600000, 0031330000, 0031320000, 0031220000, 0031210000, 0031100000))
        THEN 'Tax'
        WHEN ("glAccount" IN (0030200000))
        THEN 'Loans received'
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'SAP ERP' or "sourceSystem" = 'SAP ECP') and
    "rowType" = 'M' and
    "reportType" = 'AP' and 
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')  
    '''

data_export = export_from_RISKCUSTOM(query)    

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [3]:
# проверка на кол-во баз данных
assert len(data_export.sourceSystem.unique()) == 5

In [8]:
data_work = data_export.query('holding != "EXTERNAL"').reset_index(drop=True)
data_work = data_work[data_work.accountPurpose.isin(['Trade receivables', 'Other receivables', 'Provisions','Trade payables', 'Other payables'])]
# фильтрация по перемнной Group
data_work.currencyCode = data_work.currencyCode.replace(CCY_tech_dict) # tech ccy to normal

data_work = data_work[~(data_work.orgName.isna() & data_work.companyCode.isna())].reset_index(drop=True)
data_work['org_code'] = merge_Mapping(data_work, col='orgName')
data_work['CompCode_concat'] = data_work['companyCode'].fillna('')
data_work.loc[data_work.CompCode_concat == '','CompCode_concat'] = data_work.loc[data_work.CompCode_concat == '','org_code']
data_work['CompMap'] = merge_SalesUnits(data_work, col='CompCode_concat', merge_col='ocpSegment')

data_work['CtyCode'] = merge_Mapping(data_work, col='customerName').fillna('External')
data_work['CtyMap'] = merge_SalesUnits(data_work, col='CtyCode', merge_col='ocpSegment').fillna('External')
data_work['Cty_holding'] = merge_SalesUnits(data_work,'CtyCode',merge_col='holding').fillna('External')

data_work['Amount_USD'] = data_work['presentAmountUSD'].replace({np.NaN : 0})

data_work_pivot = data_work.pivot_table(index=['holding','customerName', 'Cty_holding', 'orgName', 'CompCode_concat', 'currencyCode', 'sourceSystem','accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','internalExternalEN','dueDate'],values=['Amount_USD','presentAmountRUB'],aggfunc=sum).reset_index()

data_work_pivot['AR_flag'] = data_work_pivot['accountPurpose']\
                                            .isin(['Trade receivables', 'Other receivables', 'Provisions'])\
                                            .astype(int)
data_work_pivot['AP_flag'] = data_work_pivot['accountPurpose']\
                                            .isin(['Trade payables', 'Other payables'])\
                                            .astype(int)

data_work_pivot['Amount_AR_USD'] = data_work_pivot['Amount_USD'] * data_work_pivot['AR_flag']
data_work_pivot['Amount_AP_USD'] = data_work_pivot['Amount_USD'] * data_work_pivot['AP_flag']

data_work_pivot = Period(data_work_pivot, day_for_count=Today, col_with_date='dueDate')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EURSALES' 'EURSALES' 'EURSALES' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA

In [13]:
data_export[data_export.holding=='SUEK'].shape[0], data_work_pivot[data_work_pivot.holding=='SUEK'].shape[0]
data_export[data_export.holding=='EUROCHEM'].shape[0], data_work_pivot[data_work_pivot.holding=='EUROCHEM'].shape[0]

(246049, 41110)

Проверка на merge:

In [5]:
check_data = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CtyMap'] == 'External'),\
                                                    ['customerName', 'CtyCode', 'CtyMap']].drop_duplicates()
check_data2 = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CompMap'] == 'External'),\
                                                    ['orgName', 'CompCode_concat','CompMap']].drop_duplicates()
check_data3 = data_work.loc[(data_work['CtyCode'] != 'External') & (data_work['Cty_holding'] == 'External'),\
                                                    ['CtyCode', 'Cty_holding']].drop_duplicates()
if print_manual_map == True:
    writer = pd.ExcelWriter('manual_map.xlsx', engine='openpyxl')  
    workbook=writer.book
    startcol = 0
    for check_data in [check_data2, check_data, check_data3]:
        check_data.to_excel(writer, index=False, startcol=startcol)
        startcol = startcol + check_data.shape[1] + 1
    writer.close()


In [6]:
BS_data = data_work_pivot[(data_work_pivot.accountPurpose.str.contains('Trade')) | (data_work_pivot.accountPurpose.str.contains('Other'))].reset_index(drop=True)
BS_data['Comp_hold'] = merge_SalesUnits(BS_data, col='CompCode_concat', merge_col='holding')
BS_data['Cpty_hold'] = merge_SalesUnits(BS_data, col='CtyCode', merge_col='holding')

for Group in ['SUEK', 'EUROCHEM']:
    BS_data_group = BS_data[(BS_data.Comp_hold == Group) & (BS_data.Cpty_hold == Group)]
    BS_data_group = BS_data_group[BS_data_group.CompMap != 'External']
    BS_data_group = BS_data_group[['CompMap', 'CtyMap', 'Amount_USD']]
    BS_pivot = pd.pivot_table(BS_data_group, values='Amount_USD', index='CompMap', aggfunc='sum', columns='CtyMap',)
    
    if Group == 'EUROCHEM':
        BS_pivot_EURO = BS_pivot
    if Group == 'SUEK':
        BS_pivot_SUEK = BS_pivot

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EUROCHEM' 'EUROCHEM' 'EUROCHEM' ... 'SUEK' 'SUEK' 'SUEK']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2

Запись в output файлы:

In [7]:
Output_SUEK = "_".join([str(date.today()), 'SUEK', 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Output_EURO = "_".join([str(date.today()), 'EUROCHEM', 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'ARAP'
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_SUEK = Output_path + Output_SUEK
    Output_EURO = Output_path + Output_EURO

if Print_ARAP == True:
    data_print_SUEK = data_work_pivot.query('holding == "SUEK"')[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                        'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                        'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]
    BS_pivot_SUEK.to_excel(Output_SUEK, sheet_name='BS_to_BS')
    new_list(data_print_SUEK, Output_SUEK, Sheet_in_output_file)
    data_print_EURO = data_work_pivot.query('holding == "EUROCHEM"')[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                        'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                        'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]
    BS_pivot_EURO.to_excel(Output_EURO, sheet_name='BS_to_BS')
    new_list(data_print_EURO, Output_EURO, Sheet_in_output_file)

In [8]:
assert 1==2

AssertionError: 

Состав ARAP

In [ ]:
data_pivot = data_work[~((data_work.AR_flag==0) & (data_work.AP_flag==0))]
group = ""
for group in ['EUROCHEM', 'SUEK']:
    data_pivot_group = data_pivot[data_pivot.holding == group]
    data_pivot_group = data_pivot_group.pivot_table(index=['holding', 'CompMap', 'CompCode_concat', 'internalExternalEN', 'AR_flag', 'AP_flag'], values='Amount_USD', aggfunc={'Amount_USD':['sum','count']}).reset_index()
    data_pivot_group.to_excel(f'{str(date.today())}_ARAP_pivot_{group}_{Mounth}.xlsx')

Поиск встречных требований:

In [4]:
# Фильт по Counterparty_Group - только внутригрупповые
data_E_to_C = data_work_pivot[(data_work_pivot.CtyMap!='External')&(data_work_pivot.Amount_USD!=0)&(data_work_pivot.accountPurpose!='Provisions')]
data_E_to_C = data_E_to_C[(data_E_to_C.AR_flag!=0)|(data_E_to_C.AP_flag!=0)]
data_E_to_C['Index_copy'] = data_E_to_C.index
data_E_to_C= data_E_to_C.fillna('_')

group_data = data_E_to_C.pivot_table(index=['CompMap', 'CompCode_concat', 'CtyMap', 'CtyCode', 'currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index_copy':list}).reset_index()
group_data['concat_col_EC'] = group_data[['CompCode_concat', 'CtyCode', 'currencyCode', 'AR_flag']].astype(str).agg('.'.join, axis=1)
group_data['concat_col_CE'] = group_data[['CtyCode', 'CompCode_concat', 'currencyCode', 'AP_flag']].astype(str).agg('.'.join, axis=1)
group_data_merge = group_data.merge(group_data, left_on='concat_col_EC', right_on='concat_col_CE', how='left')
group_data_merge = group_data_merge[group_data_merge.CompCode_concat_x==group_data_merge.CtyCode_x] # контрагент равен компании
group_data_merge = group_data_merge[['CompMap_x','CompCode_concat_x','CtyMap_x','CtyCode_x','currencyCode_x','AR_flag_x','AP_flag_x','Amount_USD_x','Index_copy_x','AR_flag_y','AP_flag_y','Amount_USD_y','Index_copy_y']]
group_data_merge = group_data_merge.fillna(0)
group_data_merge[['Amount_USD_x','Amount_USD_y']] = group_data_merge[['Amount_USD_x','Amount_USD_y']].astype(int)
# Сравнение на равенство
not_equal_data = group_data_merge[group_data_merge.Amount_USD_x.abs()!=group_data_merge.Amount_USD_y.abs()]
# To print
Index_copy_x_list = list(set(np.concatenate(not_equal_data.Index_copy_x.values.tolist())))
# Index_copy_y_list = list(set(np.concatenate(not_equal_data[not_equal_data.Index_copy_y!=0].Index_copy_y.values.tolist())))
# index_copy_list = Index_copy_x_list+Index_copy_y_list
# not_equal_data_strings = data_work.loc[index_copy_list,:]
not_equal_data_strings = data_work.loc[Index_copy_x_list,:] # контрагент равен компании
# to excel
not_equal_data = not_equal_data.rename(columns={'CompMap_x':'Segment','CompCode_concat_x':'Code','CtyMap_x':'Segmet_cpty','CtyCode_x':'Code_cty','currencyCode_x':'CCY'})

### ccy list
group_data_ccy = data_E_to_C.pivot_table(index=['currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index_copy':list}).reset_index()
group_data_ccy['concat_col_EC'] = group_data_ccy[['currencyCode', 'AR_flag']].astype(str).agg('.'.join, axis=1)
group_data_ccy['concat_col_CE'] = group_data_ccy[['currencyCode', 'AP_flag']].astype(str).agg('.'.join, axis=1)
group_data_merge_ccy = group_data_ccy.merge(group_data_ccy, left_on='concat_col_EC', right_on='concat_col_CE', how='left')
group_data_merge_ccy = group_data_merge_ccy[['currencyCode_x','AR_flag_x','AP_flag_x','Amount_USD_x','Index_copy_x','AR_flag_y','AP_flag_y','Amount_USD_y','Index_copy_y']]
group_data_merge_ccy = group_data_merge_ccy.fillna(0)
group_data_merge_ccy[['Amount_USD_x','Amount_USD_y']] = group_data_merge_ccy[['Amount_USD_x','Amount_USD_y']].astype(int)
# Сравнение на равенство
not_equal_data_ccy = group_data_merge_ccy[group_data_merge_ccy.Amount_USD_x.abs()!=group_data_merge_ccy.Amount_USD_y.abs()]

if print_counterclaims == True:
    Output_counterclaims = f'{str(date.today())}_ARAP_not_equal_counterclaims.xlsx'
    not_equal_data.to_excel(Output_counterclaims, sheet_name='pivot')
    new_list(not_equal_data_strings, Output_counterclaims, 'data', True)
    new_list(not_equal_data_ccy, Output_counterclaims, 'pivot_ccy')

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_6336\640719962.py:7: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data = data_E_to_C.pivot_table(index=['CompMap', 'CompCode_concat', 'CtyMap', 'CtyCode', 'currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index_copy':list}).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_6336\640719962.py:27: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data_ccy = data_E_to_C.pivot_table(index=['currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index

Встречные требования по валютам:

In [5]:
if print_counterclaims == True:
    writer = pd.ExcelWriter(Output_counterclaims, engine='openpyxl')  
    workbook=writer.book

# part of the data by holding
holding = ''
for holding in data_E_to_C.holding.unique().tolist():
    data_E_to_C_holding = data_E_to_C[(data_E_to_C.holding == holding) & (data_E_to_C.Cty_holding == holding)]
    startrow = 1
    currencyCode = ''
    if print_counterclaims == True:
        pd.DataFrame({'holding':f'{holding}_by_CCY'}, index=[1]).to_excel(writer, sheet_name=f'{holding}_CCY_tabels', index=False, header=False)

    # matrix for each holding
    CompMap_list = list(data_E_to_C_holding.CompMap.unique())
    CompMap_list.sort(reverse=True)
    CtyMap_list = list(data_E_to_C_holding.CtyMap.unique())
    CtyMap_list.sort(reverse=True)
    matrix_list = []
    for CompMap in CompMap_list:
        matrix_dict = {}
        matrix_dict['CompMap'] = CompMap
        for CtyMap in CtyMap_list:
            matrix_dict[CtyMap] = 0
        matrix_list.append(matrix_dict)
    matrix_df = pd.DataFrame(matrix_list)
    matrix_df.index = matrix_df['CompMap']
    matrix_df = matrix_df.drop('CompMap',axis=1)

    # part of the data by CCY
    for currencyCode in data_E_to_C_holding.currencyCode.unique().tolist():
        data_E_to_C_currencyCode = data_E_to_C_holding[data_E_to_C_holding.currencyCode == currencyCode]
        pivot = data_E_to_C_currencyCode.pivot_table(index=['CompMap','CtyMap'],values=['Amount_AR_USD','Amount_AP_USD'],aggfunc=sum).reset_index()
        
        matrix_df[:] = 0
        ccy_df = matrix_df
        for string in CompMap_list:
            status = 'AR'
            for col in CtyMap_list:
                CCY_df_part = pivot[(pivot.CompMap==string)&(pivot.CtyMap==col)]
                if string==col:
                    status = 'AP'
                    try:
                        ccy_df.loc[string, col] = CCY_df_part.Amount_AR_USD.values[0] + CCY_df_part.Amount_AP_USD.values[0]
                    except:
                        pass
                elif status == 'AP':
                    try:
                        ccy_df.loc[string, col] = CCY_df_part.Amount_AP_USD.values[0]
                    except:
                        pass
                elif status == 'AR':
                    try:
                        ccy_df.loc[string, col] = CCY_df_part.Amount_AR_USD.values[0]
                    except:
                        pass

        ### to excel
        if print_counterclaims == True:
            pd.DataFrame([{'CCY':currencyCode}]).to_excel(writer, sheet_name=f'{holding}_CCY_tabels', index=False, header=False, startrow=startrow)
            startrow += 1
            ccy_df.to_excel(writer, sheet_name=f'{holding}_CCY_tabels', index=True, startrow=startrow)
            startrow = startrow + len(ccy_df) + 2
if print_counterclaims == True:
    # data_work_pivot.to_excel(writer, sheet_name=f'data', index=False)
    writer.close()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_6336\2516715862.py:33: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = data_E_to_C_currencyCode.pivot_table(index=['CompMap','CtyMap'],values=['Amount_AR_USD','Amount_AP_USD'],aggfunc=sum).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_6336\2516715862.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '36431.46999999997' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ccy_df.loc[string, col] = CCY_df_part.Amount_AR_USD.values[0] + CCY_df_part.Amount_AP_USD.values[0]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_6336\2516715862.py:49: FutureWarning: Setting an item of incompatible dtype is deprecate

In [28]:
accountPurpose_list = ['Trade payables', 'Other payables', 'Other receivables', 'Trade receivables'] #  'Provisions'
RUB_data = data_work_pivot[(data_work_pivot.accountPurpose.isin(accountPurpose_list))&(data_work_pivot.currencyCode=='RUB')]
RUB_data_pivot = RUB_data.pivot_table(index=['holding','CompMap', 'orgName', 'CompCode_concat', 'internalExternalEN','AR_flag','AP_flag'], values=['Amount_USD','presentAmountRUB'], aggfunc='sum').reset_index()

Out_file = f'{str(date.today())}_ARAP_RUB_pivot_for_{Today}.xlsx'
writer = pd.ExcelWriter(Out_file, engine='openpyxl')
workbook = writer.book
for holding in RUB_data_pivot.holding.unique().tolist():
    RUB_data_pivot[RUB_data_pivot.holding==holding].to_excel(writer, sheet_name=holding, index=False)
writer.close()

In [23]:
# query= f"""
# select *
# from RISKACCESS."unifiedARReport"
# FETCH FIRST 3 ROWS ONLY """
# data=export_from_RISKCUSTOM(query)

,holding,customerName,Cty_holding,orgName,CompCode_concat,currencyCode,sourceSystem,accountPurpose,CompMap,CtyCode,...,internalExternalEN,dueDate,Amount_USD,presentAmountRUB,AR_flag,AP_flag,Amount_AR_USD,Amount_AP_USD,Days,Period
0,EUROCHEM,AGRO PAVAO LTDA,External,Fertilizantes Heringer,BRHRG,BRL,SAP_HRG,Trade receivables,SAMSALES,External,...,External,2024-07-18,12341.32,1.058244e+06,1,0,12341.32,0.0,18,<1M
1,EUROCHEM,ANTONIO NIDOVANDO PEREIRA PINHEIRO,External,Fertilizantes Heringer,BRHRG,BRL,SAP_HRG,Trade receivables,SAMSALES,External,...,External,2024-07-01,2351.94,2.016741e+05,1,0,2351.94,0.0,1,<1M
2,EUROCHEM,ANTONIO NIDOVANDO PEREIRA PINHEIRO,External,Fertilizantes Heringer,BRHRG,BRL,SAP_HRG,Trade receivables,SAMSALES,External,...,External,2024-07-24,2473.28,2.120788e+05,1,0,2473.28,0.0,24,<1M
3,EUROCHEM,ANTONIO NIDOVANDO PEREIRA PINHEIRO,External,Fertilizantes Heringer,BRHRG,BRL,SAP_HRG,Trade receivables,SAMSALES,External,...,External,2024-07-28,2473.35,2.120848e+05,1,0,2473.35,0.0,28,<1M
4,EUROCHEM,ANTONIO NIDOVANDO PEREIRA PINHEIRO,External,Fertilizantes Heringer,BRHRG,BRL,SAP_HRG,Trade receivables,SAMSALES,External,...,External,2024-07-31,2351.94,2.016741e+05,1,0,2351.94,0.0,31,1M-3M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258809,SUEK,한국서부발전주식회사,External,Black Sand Commodities,7B00,USD,SAP ERP,Trade receivables,SUEK INT,External,...,External,2024-02-29,12157.75,1.042503e+06,1,0,12157.75,0.0,-122,<1M
258810,SUEK,한일인터내셔널 주식회사,External,Black Sand Commodities,7B00,USD,SAP ERP,Trade receivables,SUEK INT,External,...,External,2024-02-05,-2405.90,-2.063011e+05,1,0,-2405.90,-0.0,-146,<1M
258811,SUEK,한일인터내셔널 주식회사,External,Black Sand Commodities,7B00,USD,SAP ERP,Trade receivables,SUEK INT,External,...,External,2024-03-01,9655.56,8.279450e+05,1,0,9655.56,0.0,-121,<1M
258812,SUEK,한일인터내셔널 주식회사,External,Black Sand Commodities,7B00,USD,SAP ERP,Trade receivables,SUEK INT,External,...,External,2024-03-18,3638547.00,3.119981e+08,1,0,3638547.00,0.0,-104,<1M


In [29]:
data_jun_S = pd.read_excel('2024-07-18_ARAP_RUB_pivot.xlsx', sheet_name='SUEK')
data_jun_E = pd.read_excel('2024-07-18_ARAP_RUB_pivot.xlsx', sheet_name='EUROCHEM')
data_jun = pd.concat([data_jun_S,data_jun_E])
data_jun

,holding,CompMap,orgName,CompCode_concat,internalExternalEN,AR_flag,AP_flag,Amount_USD,presentAmountRUB
0,SUEK,External,"АО ""СГК - Алтай""",External,External,0,1,-6194877.31,-5.311983e+08
1,SUEK,External,"АО ""СГК - Алтай""",External,External,1,0,4758542.79,4.080355e+08
2,SUEK,External,"АО ""СГК - Алтай""",External,Internal,0,1,-12389250.37,-1.062353e+09
3,SUEK,External,"АО ""СГК - Алтай""",External,Internal,1,0,91956.66,7.885100e+06
4,SUEK,External,"АО ""СГК - Новосибирск""",External,External,0,1,-5363919.74,-4.599454e+08
...,...,...,...,...,...,...,...,...,...
149,EUROCHEM,SPV,"ООО ""РУСБОНД-ЕВРОХИМ""",RUSBO,Internal,0,1,-21803.15,-1.869577e+06
150,EUROCHEM,TRADING,Corrigo Fertilizers LLC,AECOR,Internal,0,1,-868508.33,-7.447285e+07
151,EUROCHEM,TRADING,Corrigo Fertilizers LLC,AECOR,Internal,1,0,1100117.99,9.433292e+07
152,EUROCHEM,TRADING,EuroChem Group AG,CHEAG,External,0,1,-179.05,-1.535318e+04


In [81]:
RUB_data_merge = RUB_data_pivot.merge(data_jun, how='outer', left_on=['holding', 'CompCode_concat', 'internalExternalEN', 'AR_flag', 'AP_flag'], right_on=['holding', 'CompCode_concat', 'internalExternalEN', 'AR_flag', 'AP_flag'])\
        .rename(columns={'Amount_USD_x':'Amount_USD_dec23',	'presentAmountRUB_x':'presentAmountRUB_dec23',	'Amount_USD_y':'Amount_USD_jun24',	'presentAmountRUB_y':'presentAmountRUB_jun24'})

for col in ['CompMap', 'orgName']:
        RUB_data_merge.loc[RUB_data_merge[f'{col}_x'].isna(), f'{col}_x'] = RUB_data_merge.loc[RUB_data_merge[f'{col}_x'].isna(), f'{col}_y']
RUB_data_merge = RUB_data_merge.drop(columns=['CompMap_y', 'orgName_y'])

RUB_data_merge[['Amount_USD_dec23','presentAmountRUB_dec23','Amount_USD_jun24','presentAmountRUB_jun24']] = RUB_data_merge[['Amount_USD_dec23','presentAmountRUB_dec23','Amount_USD_jun24','presentAmountRUB_jun24']].fillna(0)

RUB_data_merge_pivot_units = RUB_data_merge.pivot_table(index=['holding','CompMap_x','internalExternalEN','AR_flag','AP_flag'], values=['Amount_USD_dec23','presentAmountRUB_dec23','Amount_USD_jun24','presentAmountRUB_jun24'], aggfunc=sum).reset_index().rename(columns={'Amount_USD_dec23':'SUM_USD_dec23','Amount_USD_jun24':'SUM_USD_jun24','presentAmountRUB_dec23':'SUM_RUB_dec23','presentAmountRUB_jun24':'SUM_RUB_jun24'})

RUB_data_merge = RUB_data_merge.merge(RUB_data_merge_pivot_units, how='left', left_on=['holding','CompMap_x','internalExternalEN','AR_flag','AP_flag'], right_on=['holding','CompMap_x','internalExternalEN','AR_flag','AP_flag'])

ccys_list = ['USD', 'RUB']
data_list = ['dec23', 'jun24']
for ccy in ccys_list:
        for dat in data_list:
                cols_list = RUB_data_merge.columns[(RUB_data_merge.columns.str.contains(ccy))&(RUB_data_merge.columns.str.contains(dat))].tolist()
                RUB_data_merge[f'part_{ccy}_{dat}'] = RUB_data_merge[cols_list[0]]/RUB_data_merge[cols_list[1]]
                RUB_data_merge[f'part_{ccy}_{dat}'] = RUB_data_merge[f'part_{ccy}_{dat}'].fillna(0)
RUB_data_merge

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18864\2166120424.py:10: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  RUB_data_merge_pivot_units = RUB_data_merge.pivot_table(index=['holding','CompMap_x','internalExternalEN','AR_flag','AP_flag'], values=['Amount_USD_dec23','presentAmountRUB_dec23','Amount_USD_jun24','presentAmountRUB_jun24'], aggfunc=sum).reset_index().rename(columns={'Amount_USD_dec23':'SUM_USD_dec23','Amount_USD_jun24':'SUM_USD_jun24','presentAmountRUB_dec23':'SUM_RUB_dec23','presentAmountRUB_jun24':'SUM_RUB_jun24'})


,holding,CompMap_x,orgName_x,CompCode_concat,internalExternalEN,AR_flag,AP_flag,Amount_USD_dec23,presentAmountRUB_dec23,Amount_USD_jun24,presentAmountRUB_jun24,SUM_USD_dec23,SUM_USD_jun24,SUM_RUB_dec23,SUM_RUB_jun24,part_USD_dec23,part_USD_jun24,part_RUB_dec23,part_RUB_jun24
0,EUROCHEM,EX SUEK AG,SUEK AG (Swiss),7100,External,0,1,0.00,0.000000e+00,-9026.25,-773982.89,0.000000e+00,-9.990300e+03,0.000000e+00,-8.566483e+05,0.000000,0.903501,0.000000,0.903501
1,EUROCHEM,EX SUEK AG,SUEK AG (Swiss),7100,Internal,0,1,0.00,0.000000e+00,-306088.22,-26246452.70,0.000000e+00,-3.060882e+05,0.000000e+00,-2.624645e+07,0.000000,1.000000,0.000000,1.000000
2,EUROCHEM,EX SUEK AG,SUEK Logistic Moscow,7Z00,External,0,1,0.00,0.000000e+00,-964.05,-82665.37,0.000000e+00,-9.990300e+03,0.000000e+00,-8.566483e+05,0.000000,0.096499,0.000000,0.096499
3,EUROCHEM,EX SUEK AG,SUEK Logistic Moscow,7Z00,External,1,0,0.00,0.000000e+00,0.01,0.86,0.000000e+00,1.000000e-02,0.000000e+00,8.600000e-01,0.000000,1.000000,0.000000,1.000000
4,EUROCHEM,EX SUEK AG,SUEK Logistic Moscow,7Z00,Internal,1,0,0.00,0.000000e+00,258225.30,22142303.03,0.000000e+00,2.582253e+05,0.000000e+00,2.214230e+07,0.000000,1.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,SUEK,SGC,"ЧОУ ДПО ""Энергоцентр""",Y600,Internal,1,0,131031.00,1.175195e+07,124061.81,10638052.11,1.621530e+08,9.088014e+07,1.454322e+10,7.792790e+09,0.000808,0.001365,0.000808,0.001365
476,SUEK,SGC,"ООО ""МС Торнадо""",Y800,External,0,1,-508776.00,-4.563125e+07,-267761.19,-22959986.54,-1.508962e+08,-7.521770e+07,-1.353363e+10,-6.449767e+09,0.003372,0.003560,0.003372,0.003560
477,SUEK,SGC,"ООО ""МС Торнадо""",Y800,External,1,0,70856.54,6.355003e+06,285949.05,24519559.15,3.160998e+08,2.274832e+08,2.835046e+10,1.950623e+10,0.000224,0.001257,0.000224,0.001257
478,SUEK,SGC,"ООО ""МС Торнадо""",Y800,Internal,0,1,-103883.72,-9.317154e+06,-8471.80,-726439.90,-2.330444e+08,-1.863314e+08,-2.090135e+10,-1.597754e+10,0.000446,0.000045,0.000446,0.000045


In [91]:
Out_file = f'{str(date.today())}_ARAP_RUB_pivot_halfyear.xlsx'
writer = pd.ExcelWriter(Out_file, engine='openpyxl')
workbook = writer.book
for holding in RUB_data_merge_pivot_units.holding.unique().tolist():
    RUB_data_merge_pivot_units[RUB_data_merge_pivot_units.holding==holding].to_excel(writer, sheet_name=holding, index=False)
    RUB_data_merge.to_excel(writer, sheet_name='data', index=False)
    part = 0.5
    RUB_data_merge.loc[(RUB_data_merge['part_RUB_dec23']>part)|(RUB_data_merge['part_RUB_jun24']>part), ['holding','orgName_x','CompCode_concat','internalExternalEN','AR_flag','AP_flag']].drop_duplicates().to_excel(writer, sheet_name='top_companies', index=False)
writer.close()


In [92]:

RUB_data_merge.loc[(RUB_data_merge['part_RUB_dec23']>part)|(RUB_data_merge['part_RUB_jun24']>part)]

,holding,CompMap_x,orgName_x,CompCode_concat,internalExternalEN,AR_flag,AP_flag,Amount_USD_dec23,presentAmountRUB_dec23,Amount_USD_jun24,presentAmountRUB_jun24,SUM_USD_dec23,SUM_USD_jun24,SUM_RUB_dec23,SUM_RUB_jun24,part_USD_dec23,part_USD_jun24,part_RUB_dec23,part_RUB_jun24
0,EUROCHEM,EX SUEK AG,SUEK AG (Swiss),7100,External,0,1,0.000000e+00,0.000000e+00,-9.026250e+03,-7.739829e+05,0.000000e+00,-9.990300e+03,0.000000e+00,-8.566483e+05,0.000000,0.903501,0.000000,0.903501
1,EUROCHEM,EX SUEK AG,SUEK AG (Swiss),7100,Internal,0,1,0.000000e+00,0.000000e+00,-3.060882e+05,-2.624645e+07,0.000000e+00,-3.060882e+05,0.000000e+00,-2.624645e+07,0.000000,1.000000,0.000000,1.000000
3,EUROCHEM,EX SUEK AG,SUEK Logistic Moscow,7Z00,External,1,0,0.000000e+00,0.000000e+00,1.000000e-02,8.600000e-01,0.000000e+00,1.000000e-02,0.000000e+00,8.600000e-01,0.000000,1.000000,0.000000,1.000000
4,EUROCHEM,EX SUEK AG,SUEK Logistic Moscow,7Z00,Internal,1,0,0.000000e+00,0.000000e+00,2.582253e+05,2.214230e+07,0.000000e+00,2.582253e+05,0.000000e+00,2.214230e+07,0.000000,1.000000,0.000000,1.000000
5,EUROCHEM,TRADING,Corrigo Fertilizers LLC,AECOR,Internal,0,1,-1.783267e+08,-1.599382e+10,-8.685083e+05,-7.447285e+07,-2.113254e+08,-3.386337e+07,-1.895342e+10,-2.903717e+09,0.843849,0.025647,0.843849,0.025647
6,EUROCHEM,TRADING,Corrigo Fertilizers LLC,AECOR,Internal,1,0,9.666560e+05,8.669774e+07,1.100118e+06,9.433292e+07,5.948163e+08,1.100118e+06,5.334806e+10,9.433292e+07,0.001625,1.000000,0.001625,1.000000
7,EUROCHEM,TRADING,Eurochem Trading ME DMCC,AETRD,Internal,0,1,-3.299870e+07,-2.959597e+09,-3.299487e+07,-2.829244e+09,-2.113254e+08,-3.386337e+07,-1.895342e+10,-2.903717e+09,0.156151,0.974353,0.156151,0.974353
9,EUROCHEM,TRADING,EuroChem Group AG,CHEAG,External,0,1,-1.790500e+02,-1.605869e+04,-1.790500e+02,-1.535318e+04,-3.995700e+02,-1.790500e+02,-3.583675e+04,-1.535318e+04,0.448107,1.000000,0.448107,1.000000
10,EUROCHEM,TRADING,EuroChem Group AG Trading,CHTRD,External,0,1,-2.205200e+02,-1.977806e+04,0.000000e+00,0.000000e+00,-3.995700e+02,-1.790500e+02,-3.583675e+04,-1.535318e+04,0.551893,-0.000000,0.551893,-0.000000
11,EUROCHEM,TRADING,EuroChem Group AG Trading,CHTRD,Internal,1,0,5.938496e+08,5.326136e+10,0.000000e+00,0.000000e+00,5.948163e+08,1.100118e+06,5.334806e+10,9.433292e+07,0.998375,0.000000,0.998375,0.000000
